In [1]:
#! pip install transformers datasets accelerate scikit-learn -Uqq

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import numpy as np
import pandas as pd
import torch
from functools import partial
import gc
from huggingface_hub import login


from datasets import load_dataset
from sklearn.metrics import f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    TrainerCallback,
)

from sklearn.metrics import matthews_corrcoef, accuracy_score, f1_score
from scipy.stats import pearsonr, spearmanr

os.environ["TOKENIZERS_PARALLELISM"] = "false"

hf_token = "teste"
login(hf_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/vbertalan/.cache/huggingface/token
Login successful


In [11]:
checkpoint = "answerdotai/ModernBERT-base"  # "answerdotai/ModernBERT-base", "answerdotai/ModernBERT-large"

In [16]:
path = "/home/vbertalan/.dataset/"
dataset = "hdfs_2k/HDFS_2k.log"

full_path = path + dataset

import os

if os.path.isfile(full_path):
    print("arquivo existe")

## Tokenização

hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=hf_token)

def preprocess_function(examples, task_inputs):
    inps = [examples[inp] for inp in task_inputs]
    tokenized = hf_tokenizer(*inps, truncation=True)
    return tokenized

text = "Hello there"
enc = hf_tokenizer.encode_plus(text)
enc.keys()
print(enc)


arquivo existe
{'input_ids': [50281, 12092, 627, 50282], 'attention_mask': [1, 1, 1, 1]}


/home/vbertalan/anaconda3/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [19]:
import torch
from transformers import AutoTokenizer, AutoModel


# Load tokenizer
hf_tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=hf_token)

# Load model
model = AutoModel.from_pretrained(checkpoint, use_auth_token=hf_token)

# Put model in evaluation mode
model.eval()

# Example text
text = "Hello there"

# Tokenize input
inputs = hf_tokenizer(text, return_tensors="pt")

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract hidden states (embeddings)
embeddings = outputs.last_hidden_state  # Shape: [batch_size, seq_length, hidden_dim]

# Get a single vector representation (e.g., mean pooling)
sentence_embedding = embeddings.mean(dim=1)  # Shape: [batch_size, hidden_dim]

print("Embedding shape:", sentence_embedding.shape)
print(sentence_embedding)


/home/vbertalan/anaconda3/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/vbertalan/anaconda3/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


ValueError: The checkpoint you are trying to load has model type `modernbert` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.